In [13]:
from langchain_groq import ChatGroq

In [14]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("Who was the first prime minister of india was...")
print(response.content)

The first Prime Minister of India was Jawaharlal Nehru. He served as the Prime Minister from August 15, 1947, to May 27, 1964.


In [15]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.mercedes-benz.com/en/lead-full-stack-developer-136292-MER0003ATN")
page_data = loader.load().pop().page_content
print(page_data)





Lead Full Stack Developer | Mercedes-Benz > Career > Job Search > Job Postings





































































 DE  EN Provider/PrivacyCompanyInnovationResponsibilityCareersInvestorsPressProductsCareersJob searchLead Full Stack DeveloperJob category:Research and Development incl. DesignDepartment:Data Engineering 2Organization:Mercedes-Benz Research and Development India Private LimitedLocation:Mercedes-Benz Research and Development India, BangaloreStart date:immediatelyPublication date:17.09.2024Job number:MER0003ATNWorking time:full time ApplyTasksWe are looking for a Full Stack Developer to produce scalable fast performing web application to display high frequency data without experiencing any performance degradations.  

You’ll be part of a team that’s responsible for the full software development life cycle, from conception to deployment. 
You should be comfortable around both front-end and back-end coding languages, development frameworks 

In [16]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [17]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Full Stack Developer',
 'experience': 'Not specified',
 'skills': ['HTML/CSS',
  'JavaScript',
  'XML',
  'jQuery',
  'Angular',
  'React',
  'Node.js',
  'API development',
  'Web sockets IO',
  'SQL',
  'Postgres',
  'C#',
  'Java',
  'Python'],
 'description': 'We are looking for a Full Stack Developer to produce scalable fast performing web application to display high frequency data without experiencing any performance degradations. You’ll be part of a team that’s responsible for the full software development life cycle, from conception to deployment.'}

In [18]:
type(json_res)

dict

In [19]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [20]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [21]:
links = collection.query(query_texts=["Experience in Python","Experience in Azure"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}]]

In [22]:
job=json_res
job['skills']

['HTML/CSS',
 'JavaScript',
 'XML',
 'jQuery',
 'Angular',
 'React',
 'Node.js',
 'API development',
 'Web sockets IO',
 'SQL',
 'Postgres',
 'C#',
 'Java',
 'Python']

In [23]:
job = json_res
job['skills']

['HTML/CSS',
 'JavaScript',
 'XML',
 'jQuery',
 'Angular',
 'React',
 'Node.js',
 'API development',
 'Web sockets IO',
 'SQL',
 'Postgres',
 'C#',
 'Java',
 'Python']

In [24]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Dev, a business development executive at Mercedes-Benz. Mercedes is
        a leading innovator in the automotive industry and is exploring the integration of
        cutting-edge technology into its vehicles and internal operations. Over our
        experience, we have empowered our partners with high-performance solutions,
        optimizing speed, reliability, and efficiency in both our products and processes.

        Your job is to write a cold email to the client regarding the job mentioned above describing 
        the capability of Mercedes in fulfilling their needs. Also, add the most relevant ones from 
        the following links to showcase Mercedes' portfolio: {link_list}.

        Remember you are Dev, BDE at Mercedes-Benz.
        ### EMAIL (NO PREAMBLE):
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list":links})
print(res.content)

Subject: Unlocking High-Performance Solutions for Your Web Application

Dear Hiring Manager,

I came across your job posting for a Lead Full Stack Developer and was impressed by your company's vision for a scalable and fast-performing web application. As a Business Development Executive at Mercedes-Benz, I'd like to introduce you to our team's capabilities in delivering high-performance solutions that can meet your needs.

At Mercedes-Benz, we've empowered our partners with cutting-edge technology, optimizing speed, reliability, and efficiency in both our products and processes. Our expertise in software development, particularly in full-stack development, can help you achieve your goals. Our team has experience working with a range of technologies, including HTML/CSS, JavaScript, Node.js, and SQL, which aligns with your job requirements.

I'd like to highlight a few examples of our work that demonstrate our capabilities:

* Our Python expertise has been showcased in projects such as [